<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект-2: Анализ вакансий из HeadHunter
   

In [23]:
# Импортируем необходимые нам библиотеки

import pandas as pd
import psycopg2

# Игнорируем всплывающие предупрежедения 
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

In [25]:
# Подключаемся к нашей базе данных для выполнения запросов из ноутбука

connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

## Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [26]:
# Считаем количество записей в таблице 'vacancies'

query_3_1 = f'''
SELECT
     COUNT(*) vacancies_cnt
FROM
     vacancies
'''

In [27]:
df = pd.read_sql_query(query_3_1, connection)
df

,vacancies_cnt
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [28]:
# Считаем количество записей в таблице 'employers'

query_3_2 = f'''
SELECT
     COUNT(*) employers_cnt
FROM
     employers
'''

In [29]:
df = pd.read_sql_query(query_3_2, connection)
df

,employers_cnt
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [30]:
# Считаем количество записей в таблице 'areas'

query_3_3 = f'''
SELECT
     COUNT(*) areas_cnt
FROM
     areas
'''

In [31]:
df = pd.read_sql_query(query_3_3, connection)
df

,areas_cnt
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [32]:
# Считаем количество записей в таблице 'industries'

query_3_4 = f'''
SELECT
     COUNT(*) industries_cnt
FROM
     industries
'''

In [33]:
df = pd.read_sql_query(query_3_4, connection)
df

,industries_cnt
0,294


***

Из предварительного анализа наших данных мы можем увидеть, что среди более чем `49 тысяч вакансий` у нас имеются свыше `23 тысяч работодателей` в `1362 регионах` с `294 сферами деятельности`.

Такое количество вакансий, работодателей, уникальных регионов и разнообразных сфер деятельности работодателей говорит о том, что:

* Рынок труда имеет `положительную тенденцию` по отношению к сфере *IT*, ввиду явной `заинтересованности самих работодателей` в увеличении штата *IT* сотрудников.

Далее мы проведём детальный анализ вакансий, чтобы узнать следующие подробности:

* В каких регионах вакансий много;

* Как часто работодатели заполняют хотя бы одно из двух полей с зарплатой;

* Разберём значения для нижней и верхней границы зарплатной вилки;

* Какое количество вакансий имеется для каждого сочетания типа рабочего графика и типа трудоустройства, а так же требуемого опыта работы.

## Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [34]:
# Считаем количество вакансий для каждого региона, присоединив к таблице 'areas' таблицу 'vacancies'
# Группируем данные по названию региона, а также сортируем данные по количеству вакансий 
# Выводим ТОП-5 регионов по количеству вакансий в нём

query_4_1 = f'''
SELECT
     a.name area_name,
     COUNT(*) vacancies_cnt
FROM
     areas a
JOIN vacancies v ON v.area_id = a.id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5
'''

In [35]:
df = pd.read_sql_query(query_4_1, connection)
df

,area_name,vacancies_cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [36]:
# Считаем количество записей в таблице 'vacancies', где заполнено хотя бы одно из двух полей с зарплатой

query_4_2 = f'''
SELECT
     COUNT(*) vacancies_cnt
FROM
     vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [37]:
df = pd.read_sql_query(query_4_2, connection)
df

,vacancies_cnt
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [38]:
# Считаем средние значения для нижней и верхней границы ЗП в таблице 'vacancies'
# Получившиеся значения округляем до целого числа

query_4_3 = f'''
SELECT
     ROUND(AVG(salary_from)) avg_salary_from,
     ROUND(AVG(salary_to)) avg_salary_to
FROM
     vacancies
'''

In [39]:
df = pd.read_sql_query(query_4_3, connection)
df

,avg_salary_from,avg_salary_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.

In [40]:
# Выводим сочетания типа рабочего графика и типа трудоустройства, а также количество вакансий для каждой связки
# Группируем данные по типу рабочего графика и типу трудоустройства
# Сортируем данные по количеству вакансий для связки

query_4_4 = f'''
SELECT
     schedule schedule_type,
     employment employment_type,
     COUNT(*) vacancies_cnt
FROM
     vacancies
GROUP BY 1, 2
ORDER BY 3 DESC
'''

In [41]:
df = pd.read_sql_query(query_4_4, connection)
df

,schedule_type,employment_type,vacancies_cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [42]:
# Выводим значения опыта работы и количество вакансий для каждого из них
# Группируем данные по опыту работы и сортируем по количеству вакансий

query_4_5 = f'''
SELECT
     experience,
     COUNT(*) vacancies_cnt
FROM
     vacancies
GROUP BY 1
ORDER BY 2
'''

In [43]:
df = pd.read_sql_query(query_4_5, connection)
df

,experience,vacancies_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Подводя итог детальному анализу вакансий, можно выделить следующие моменты:

1. `Пятёрку лидеров` по количеству вакансий занимают следующие города:

   * Москва — Санкт-Петербург — Минск — Новосибирск — Алматы
  
    Разница между столицей (`Москва`) и остальными регионами обуславливается количеством населения (более чем в 2 раза по сравнению с Санкт-Петербургом), что в свою очередь увеличивает количество потенциальных работодателей и количество вакансий в регионе;

2. `У менее, чем половины` предлагаемых вакансий не указана верхняя и нижняя граница заработной платы. Такое решение работодателей во многом зависит от уровня кандидата — если опытного кандидата в процессе переговоров что-то не устроит, то придётся сделать предложение человеку с меньшим опытом. Разумеется, увидев вилку зарплат, он будет рассчитывать на максимум, и получив предложение с гораздо меньшим окладом, разочаруется и тоже может отказаться;

3. `Средние значения` для нижней и верхней границы зарплатной вилки в размере 71065 и 110537 тысяч рублей соответственно характеризуются спросом на квалифицированных специалистов, что становятся всё более ценными. Следовательно компании, готовые конкурировать за лучших сотрудников готовы предлагать и более высокие границы зарплатной вилки, чтобы привлечь и удержать таланты;

4. Помимо очевидного первенства сочетания `Полный день — Полная занятость` (более 70%) в типах рабочего графика и трудоустройства, стоит также отметить и связку `Удалённая работа — Полная занятость`, что находится на втором месте. Её популярность по сравнению с остальными связками характеризуется возможностью осуществлять профессиональную деятельность сотрудника из любого уголка мира. Ведь сотрудник может работать где угодно, если это место предполагает наличие стабильного интернета и электросети. А значит и вакансия, предлагаемая работодателем располагает к выполнению задач удалённо, без необходимости в постоянном посещении офиса;

5. Среди `Опыта работы` можно выделить заинтересованность работодателей в кандидатах с опытом работы `От 1 года до 3 лет`. Такой интерес можем быть вызван следующими факторами:

   1. Переобучить чужую `«звезду»` может быть долго, дорого и некогда, в отличие от новичка которого всегда можно с лёгкостью переобучить или доучить, а деньги, которые компания вложила в обучение для `«выращивания»` собственных
  кадров, окупятся с большей вероятностью;

   2. Сотрудники с меньшим опытом ожидаемо задают множество вопросов. Они не знают всех тонкостей работы, с энтузиазмом пробуют новое и ошибаются чаще других. Правильный новичок жаждет знаний и ищет любые способы поскорее повысить квалификацию, что также является плюсом как для самого сотрудника, так и работодателя.

Теперь перейдём к анализу самих нанимателей. Посмотрим на лидеров по количеству вакансий, регионы в которых они публикуют свои предложения и их сферы деятельности.

## Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [44]:
# Выводим работодателей, которые находятся на первом и пятом месте по количеству вакансий
# К таблице 'employers' присоединяем таблицу 'vacancies' 
# Группируем вакансии по названию работодателя и сортируем по количеству вакансий
# Лимит для вывода устанавливаем согласно необходимым местам в рейтинге  

query_5_1 = f'''
(SELECT
     e.name employer_name
FROM
     employers e
JOIN vacancies v on v.employer_id = e.id
GROUP BY 1
ORDER BY COUNT(*) DESC
LIMIT 1)
UNION ALL
(SELECT
     e.name employer_name
FROM
     employers e
JOIN vacancies v on v.employer_id = e.id
GROUP BY 1
ORDER BY COUNT(*) DESC
LIMIT 1 OFFSET 4)
'''

In [45]:
df = pd.read_sql_query(query_5_1, connection)
df

,employer_name
0,Яндекс
1,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [46]:
# Выводим название региона, количество работодателей и вакансий в нём
# Для этого к таблице 'areas' присоединяем таблицы 'employers' и 'vacancies'
# Оставляем только те регионы, где нет вакансий
# Для оставшихся записей производим группировку по ID региона и сортировку по количеству работодателей
# Лимит для вывода устанавливаем согласно необходимым требованиям   

query_5_2 = f'''
SELECT
     a.name region_name,
     COUNT(DISTINCT e.id) employers_cnt,
     COUNT(DISTINCT v.id) vacancies_cnt
FROM
     areas a
LEFT JOIN employers e on e.area = a.id
LEFT JOIN vacancies v on v.area_id = a.id
WHERE v.id IS NULL
GROUP BY a.id
ORDER BY 2 DESC
LIMIT 1
'''

In [47]:
df = pd.read_sql_query(query_5_2, connection)
df

,region_name,employers_cnt,vacancies_cnt
0,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [48]:
# Выводим название компании и количество регионов, в которых она публикует свои вакансии
# Для этого присоединяем к таблице 'vacancies' таблицу 'employers'
# Данные группируем по названию компании и сортируем по количеству регионов

query_5_3 = f'''
SELECT
     e.name employer_name,
     COUNT(DISTINCT v.area_id)
FROM
     vacancies v
JOIN employers e ON e.id = v.employer_id
GROUP BY 1
ORDER BY 2 DESC
'''

In [49]:
df = pd.read_sql_query(query_5_3, connection)
df

,employer_name,count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [50]:
# Считаем количество записей получившихся после присоединения таблиц 'employers' и 'employers_industries' по необходимому условию

query_5_4 = f'''
SELECT
     COUNT(*) employers_cnt
FROM
     employers e
LEFT JOIN employers_industries ei on ei.employer_id = e.id
WHERE ei.industry_id is NULL
'''

In [51]:
df = pd.read_sql_query(query_5_4, connection)
df

,employers_cnt
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [52]:
# Выводим название компании
# Присоединяем к таблице 'employers' таблицу 'employers_industries' и избавляемся от пропусков
# Оставляем только те записи, где количество сфер деятельности равно 4
# Сортируем данные по названию компании в алфавитном порядке
# Лимит вывода устанавливаем согласно необходимым требованиям

query_5_5 = f'''
SELECT
     e.name employer_name
FROM
     employers e
LEFT JOIN employers_industries ei on ei.employer_id = e.id
WHERE ei.industry_id IS NOT NULL
GROUP BY 1
HAVING COUNT(*) = 4
ORDER BY 1
LIMIT 1 OFFSET 2
'''

In [53]:
df = pd.read_sql_query(query_5_5, connection)
df

,employer_name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [54]:
# Считаем количество получившихся записей после присоединения таблиц 'employers_industries' и 'industries' по необходимому условию
# Для корректного подсчёта данные группируем по целевому названию сферы деятельности 

query_5_6 = f'''
SELECT
     COUNT(*) employer_cnt
FROM
     employers_industries ei
JOIN industries i ON i.id = ei.industry_id
WHERE ei.industry_id IS NOT NULL
AND i.name LIKE 'Разработка программного обеспечения'
GROUP BY i.name
'''

In [55]:
df = pd.read_sql_query(query_5_6, connection)
df

,employer_cnt
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [56]:
# Импортируем необходимые нам для нахождения списка городов-миллионников библиотеки

import requests 
from bs4 import BeautifulSoup 

# Задаём ссылку на страницу с таблицей нужных нам городов

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России' 
response = requests.get(url)

# Далее работа с содержимым страницы

page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup, указывая html-парсер
print(page.title.text,'\n')               
tabl = page.find("table", class_="standard")
million_cities = []
tt = tabl.find("tbody") # В таблице выбираем тело таблицы
qq = tt.find_all("tr")  # В теле таблицы выбираем все ряды

for ss in qq:
  qqq = list(ss.find("a"))              # В каждом ряде находим тег "а", преобразуем его в список,
  million_cities.append(qqq[0])         # и добавляем содержимое к списку городов
million_cities = million_cities[1:]     # В первом ряду кусок от названия, отсекаем его
million_cities = tuple(million_cities)  # Получившийся список переводим в кортеж
print(million_cities)                   # Проверяем результат на корректность названий и наличие всех необходимых городов

Города-миллионеры России — Википедия 

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Красноярск', 'Нижний Новгород', 'Челябинск', 'Уфа', 'Самара', 'Ростов-на-Дону', 'Краснодар', 'Омск', 'Воронеж', 'Пермь', 'Волгоград')


In [57]:
# Создаём новую таблицу с названиями городов-миллионников и количеством вакансий в них 
# Для этого объединяем таблицу 'vacancies', 'employers' и 'areas'
# Оставляем только вакансии компании "Яндекс", находящиеся в наших городах-миллионниках "million_cities"
# Чтобы подсчитать количество вакансий в каждом городе данные необходимо сгруппировать по названию города

# Из получившейся таблицы выводим знакомые нам названия городов и количество вакансий в них
# Также нам понадобится «счётчик» 'Total' значением которого будет общее количество вакансий во всех городах
# Вывод сортируем по возрастанию количества вакансий

query_5_7 = f'''
WITH towns AS (
SELECT
     a.name,
     COUNT(v.id) cnt
FROM
     vacancies v
JOIN employers e ON e.id = v.employer_id
JOIN areas a ON a.id = v.area_id
WHERE e.name = 'Яндекс'
AND a.name IN {million_cities}
GROUP BY a.name
)

SELECT
     towns.name area_name,
     towns.cnt vacancies_cnt
FROM
     towns
UNION ALL
SELECT
     'Total', 
     sum(towns.cnt)  
FROM
     towns
ORDER BY 2
'''

In [58]:
df = pd.read_sql_query(query_5_7, connection)
df

,area_name,vacancies_cnt
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Самара,26.0
8,Уфа,26.0
9,Краснодар,30.0


***

Поговорим о том, что нам дал анализ работодателей:

1. `Безусловным лидером` по количеству опубликованных вакансий является компания `"Яндекс"`, что по совместительству входит в список одних из крупнейших IT-компаний в России. Учитывая рост как самой компании, так и её количества сервисов очевидна потребность в увеличении штата сотрудников. Она же является и лидером по количеству регионов, в которых компания публикует свои вакансии — 181 регион, охватывая при этом все города-миллионники России;

2. В наших данных имеются работодатели, у которых `не указана сфера деятельности`. У таких компаний меньший шанс попасть в рекомендацию к соискателю что ищет компании по фильтру `"Отрасль компании"` или попасть в подборку тех, что могли бы его заинтересовать. Возможно, такие компании делают акцент на свой департамент *HR*, что самостоятельно откликаются на подходящие по их мнению резюме и уже во время звонка-интервью рассказывают кандидату подробности о компании;

3. `Самой популярной` сферой деятельности среди компаний является `"Разработка программного обеспечения"`. Она же и сама пополуряная инженерная дисциплина IT в принципе, ведь отвечает как за функционирование, так и за сопровождение всех программных продуктов компании.

Теперь мы подойдём к нашим данным со стороны предметного анализа — анализа вакансий, имеющих отношение к данным. Разберём их количество, требуемые ключевые навыки для кандидатов и среднюю заработную плату для каждого типа требуемого опыта работы.

## Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [59]:
# Считаем все вакансии, названия которых имеют отношение к данным

query_6_1 = f'''
SELECT
     COUNT(*) vacancies_cnt
FROM
     vacancies
WHERE name ILIKE '%data%'
OR name ILIKE '%данн%'
'''

In [60]:
df = pd.read_sql_query(query_6_1, connection)
df

,vacancies_cnt
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [61]:
# Создаём новую таблицу — 'ds' с вакансиями для дата-сайентиста
# Для этого записи в таблице 'vacancies' сортируем по вышеупомянутым условиям 

# В получившейся таблице считаем количество вакансий для начинающего дата-сайентиста
# Для подсчёта используем критерии указанные выше

query_6_2 = f'''
WITH ds AS (
SELECT
     *
FROM
     vacancies
WHERE (name ILIKE '%data scientist%'
OR name ILIKE '%data science%'
OR name ILIKE '%исследователь данных%'
OR name ILIKE '%ML%'
OR name ILIKE '%machine learning%'
OR name ILIKE '%машинн%обучен%')
AND name NOT ILIKE '%HTML%'
)

SELECT
     COUNT(*) vacancies_cnt
FROM
     ds
WHERE name ILIKE '%junior%'
OR experience ILIKE '%Нет опыта%'
OR employment ILIKE '%Стажировка%'
'''

In [62]:
df = pd.read_sql_query(query_6_2, connection)
df

,vacancies_cnt
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [63]:
# В таблице 'ds' считаем количество вакансий c целевыми ключевыми навыками указанными выше 

query_6_3 = f'''
WITH ds AS (
SELECT
     *
FROM
     vacancies
WHERE (name ILIKE '%data scientist%'
OR name ILIKE '%data science%'
OR name ILIKE '%исследователь данных%'
OR name ILIKE '%ML%'
OR name ILIKE '%machine learning%'
OR name ILIKE '%машинн%обучен%')
AND name NOT ILIKE '%HTML%'
)

SELECT
     COUNT(*) vacancies_cnt
FROM
     ds
WHERE key_skills ILIKE '%SQL%'
OR key_skills ILIKE '%postgres%'
'''

In [64]:
df = pd.read_sql_query(query_6_3, connection)
df

,vacancies_cnt
0,229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [65]:
# Считаем количество вакансий в таблице 'ds' c ключевым навыком "Python" 

query_6_4 = f'''
WITH ds AS (
SELECT
     *
FROM
     vacancies
WHERE (name ILIKE '%data scientist%'
OR name ILIKE '%data science%'
OR name ILIKE '%исследователь данных%'
OR name ILIKE '%ML%'
OR name ILIKE '%machine learning%'
OR name ILIKE '%машинн%обучен%')
AND name NOT ILIKE '%HTML%'
)

SELECT
     COUNT(*) vacancies_cnt
FROM
     ds
WHERE key_skills ILIKE '%Python%'
'''

In [66]:
df = pd.read_sql_query(query_6_4, connection)
df

,vacancies_cnt
0,357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [67]:
# Считаем среднее количество ключевых навыков в вакансиях для дата-сайентиста, где они указаны
# Полученное число округляем до двух знаков после точки-разделителя

query_6_5 = f'''
WITH ds AS (
SELECT
     *
FROM
     vacancies
WHERE (name ILIKE '%data scientist%'
OR name ILIKE '%data science%'
OR name ILIKE '%исследователь данных%'
OR name LIKE '%ML%'
OR name ILIKE '%machine learning%'
OR name ILIKE '%машинн%обучен%')
AND name NOT ILIKE '%HTML%'
)

SELECT
     round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2) avg_key_skills
FROM
     ds
WHERE key_skills IS NOT NULL
'''

In [68]:
df = pd.read_sql_query(query_6_5, connection)
df

,avg_key_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [69]:
# Из таблицы с вакансиях для дата-сайентиста выводим значения опыта работы и среднюю ЗП для каждой категории, округлённую до целого числа
# Отсекаем все вакансии, где не указана верхняя и нижняя граница уровня заработной платы 
# Для корректного подсчёта, записи группируем по категориям опыта работы

query_6_6 = f'''
WITH ds AS (
SELECT
     *
FROM
     vacancies
WHERE (name ILIKE '%data scientist%'
OR name ILIKE '%data science%'
OR name ILIKE '%исследователь данных%'
OR name LIKE '%ML%'
OR name ILIKE '%machine learning%'
OR name ILIKE '%машинн%обучен%')
AND name NOT ILIKE '%HTML%'
)

SELECT
     experience,
     ROUND(AVG(COALESCE((salary_to + salary_from) / 2, salary_from, salary_to)), 0) avg_salary
FROM
     ds
WHERE salary_from IS NOT NULL
OR salary_to IS NOT NULL
GROUP BY 1
'''

In [70]:
df = pd.read_sql_query(query_6_6, connection)
df

,experience,avg_salary
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

Из предметного анализа наших данных, мы узнали, что:

1. Из общего количества вакансий только 1771 `имеет отношение к данным`, что может говорить о большой конкуренции среди соискателей. Из них 51 вакансия подойдёт для специалистов уровня *`Junior`*, что ещё раз указывает на возможную конкуренцию среди начинающих специалистов;

2. В 229 вакансиях в качестве ключевого навыка указан *`SQL`* или *`postgres`*. В 357 указан *`Python`*. Очевидна популярность и явная необходимость вышеперечисленных языков программирования для специалиста *`Data Science`*.
 
    В среднем же ключевых навыков в вакансиях для *`DS`* c учётом вышеперечисленных языков программирования необходимо 6, как указывают работодатели. Компании ожидают от специалиста знаний и опыта, которые помогут выполнять работу эффективнее и быстрее добиваться результата в работе с разными структурами и данными;

3. Для специалистов *`DS`* наблюдается быстрый карьерный рост — интенсивность роста заработной платы практически в 2 раза в зависимости от опыта работы, от 76 до 243 тысяч рублей за 3 года!

    Это показывает насколько сильно востребованы компаниями высококвалифицированные *`DS`* специалисты.

Далее мы проведём заключительный этап нашего проекта — дополнительные исследования для полноты выводов, и наконец подведём итоги проделаной работе. 

## Дополнительные исследования

### Каких вакансий много?

In [71]:
# Выводим ТОП-5 вакансий по их количеству и их средний уровень ЗП из таблицы 'vacancies'
# Оставляем только те записи, где указана верхняя или нижняя граница уровня заработной платы
# Группируем данные по названию вакансии, а сортируем по их количеству
# Лимит вывода устанавливаем согласно ТОП'у (ввиду несовпадения в указании профессий "Программист 1С" увеличим вывод на 1)

query = f'''
SELECT DISTINCT
     name vacancy_name,
     COUNT(*) vacancies_cnt,
     ROUND(AVG(COALESCE((salary_to + salary_from) / 2, salary_from, salary_to)), 0) avg_salary
FROM
     vacancies
WHERE salary_from IS NOT NULL
OR salary_to IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 6
'''

In [72]:
df = pd.read_sql_query(query, connection)
df

,vacancy_name,vacancies_cnt,avg_salary
0,Системный администратор,1287,50115.0
1,Программист 1С,729,110189.0
2,Специалист технической поддержки,335,43211.0
3,Инженер-программист,279,50478.0
4,Программист 1C,228,96906.0
5,Аналитик,197,61197.0


### Сколько ключевых навыков в среднем указывают в вакансиях для DS с разным опытом работы?

In [73]:
# Из таблицы с данными о вакансиях для дата-сайентиста выводим категории опыта работы и среднее количество ключевых навыков для каждой из них
# Избавляемся от записей, где не указаны ключевые навыки
# Группируем данные по опыту работы. Сортировку проводим по возрастанию среднего количества ключевых навыков

query = f'''
WITH ds AS (
SELECT
     *
FROM
     vacancies
WHERE (name ILIKE '%data scientist%'
OR name ILIKE '%data science%'
OR name ILIKE '%исследователь данных%'
OR name LIKE '%ML%'
OR name ILIKE '%machine learning%'
OR name ILIKE '%машинн%обучен%')
AND name NOT ILIKE '%HTML%'
)

SELECT
     experience,
     round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) + 1), 2) avg_key_skills
FROM
     ds
WHERE key_skills IS NOT NULL
GROUP BY 1
ORDER BY 2
'''

In [74]:
df = pd.read_sql_query(query, connection)
df

,experience,avg_key_skills
0,Более 6 лет,5.94
1,От 3 до 6 лет,5.99
2,От 1 года до 3 лет,6.73
3,Нет опыта,7.42


### Что можно сказать об уровнях ЗП в разных городах-миллионниках России?

In [75]:
# Выводим название города-миллионника и его уровень (среднее значение) ЗП
# Для этого к таблице 'areas' присоединяем таблицу 'vacancies'
# Проверяем заполненность границ зарплатной вилки и вхождение города в кортеж "million_cities"
# Для подсчёта данные группируем по названию города, а вывод сортируем по уровню зарплаты в порядке убывания 

query = f'''
SELECT
     a.name area_name,
     ROUND(AVG(COALESCE((v.salary_to + v.salary_from) / 2, v.salary_from, v.salary_to)), 0) avg_salary
FROM
     areas a
JOIN vacancies v ON v.area_id = a.id
WHERE (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)
AND a.name IN {million_cities}
GROUP BY 1
ORDER BY 2 DESC
'''

In [76]:
df = pd.read_sql_query(query, connection)
df

,area_name,avg_salary
0,Москва,134140.0
1,Новосибирск,107777.0
2,Санкт-Петербург,104410.0
3,Нижний Новгород,96059.0
4,Пермь,95985.0
5,Ростов-на-Дону,94065.0
6,Краснодар,92328.0
7,Казань,92207.0
8,Екатеринбург,91896.0
9,Омск,86393.0


Проведя дополнительный анализ данных, мы можем дополнить наши выводы следующими наблюдениями:

1. `Самыми востребованными` специалистами по количеству вакансий являются `"Системные администраторы"` — при относительно небольшой средней ЗП эти специалисты нужны в любой компании, где сотрудники работают на компьютерах, ведь они устанавливают, настраивают и поддерживают бесперебойную работу и безопасность *IT*-систем: приложений, сайтов, локальной сети и всего софта в компании. От «сисадмина» будет зависеть, насколько комфортно будет работать команде, будет ли стабильной внутренняя технологическая инфраструктура, как быстро будут устраняться неполадки и насколько оперативно восстанавливается инфраструктура после аварий;

   Потому очевидна их необходимость в компаниях, следственно как и количество вакансий на данную должность.

2. В среднем для вакансий на роль *Data Scientist* `указывают разное количество ключевых навыков` с разным опытом работы. Тяжелее всего приходится новичкам — категория `"Нет опыта"`, где среднее количество ключевых навыков равно 7.42 (что в идеале равно 8). Это подтверждает наш предыдущий вывод из этапа `Предметный анализ` — компании ожидают от специалиста знаний и опыта, которые помогут выполнять работу эффективнее и быстрее добиваться результата в работе с разными структурами и данными.

   С увеличением же опыта работы количество необходимых навыков заметно понижается, вплоть до 6. Вероятнее всего это связано с уровнем знания этих самых ключевых навыков кандидатом. Глубокие познания определённых языков программирования позволяют специалисту отлично выполнять свою работу с имеющимися «инструментами» не прибегая к использованию сторонних структур, а потому и необходимость в них отпадает;

3. Смотря на уровень ЗП *IT*-специалистов в разных городах-миллионниках России можно отчётливо увидеть какие регионы являются `«движущей силой»` разивития *IT* в России. Об этом нам говорит средний уровень заработной платы, что указывает на большую востребованность в высококвалифицированных и сложных самих по себе профессий.

## Общий вывод по проекту

Подводя итоги всей проделанной работы над данными можно сказать, что рынок труда имеет `положительную динамику` роста вакансий по отношению к сфере *IT* в самых разных сферах деятельности компаний, ввиду как явной `заинтересованности самих компаний` в увеличении штата *IT* сотрудников, так и скорости развития самих *IT* технологий в целом.

Отдельно можно отметить:

1. `Достойный уровень` заработной платы во всех городах-миллионниках России;

2. `Большое общее количество` вакансий в крупных городах и регионах;

3. Необходимость в специалистах `«всех мастей»` и количество вакансий на любой вкус, включая предполагаемый тип трудоустройства и занятости;

4. `Наличие` работодателей с количеством сфер деятельности больше одной;

5. `Размер` самих компаний-нанимателей.

Всё вышеперечисленное ещё раз указывает нам на то, что рынок труда *IT* в ближайшие пару лет не будет «стоять на месте», количество вакансий будет лишь увеличиваться, как и потребность работодателей в самых разных специалистах сферы *IT*.

In [77]:
# Не забываем закрыть соединение после окончания работы

connection.close()